In [6]:
#import packages and check environment
## feature generation
import pandas as pd
from tqdm import tqdm
from itertools import combinations

## models
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
import numpy as np
import joblib 
import os 


print('hello')

hello


In [3]:
train = pd.read_csv('train.csv')
train.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [4]:
df_train = train[train['stock_id']==0].copy() # need a copy before modify the original df
df_train.shape

(26455, 17)

In [5]:
def generate_features(df):
    features = ['seconds_in_bucket'
                , 'imbalance_buy_sell_flag'
                , 'imbalance_size'
                , 'matched_size'
                , 'bid_size'
                , 'ask_size'
                , 'reference_price'
                ,'far_price'
                , 'near_price'
                , 'ask_price'
                , 'bid_price'
                , 'wap'
                , 'imb_s1'
                , 'imb_s2']
# use the formula instead of the column function
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']

    print("Generating 1st set of features...")
    for p1, p2 in tqdm(combinations(prices, 2)):
        df[f'{p1}_{p2}_imb'] = df.eval(f'({p1} - {p2})/({p1} + {p2})')
        features.append(f'{p1}_{p2}_imb')
    

    print("Generating 2st set of features...")
    for p1, p2, p3 in tqdm(combinations(prices, 3)):
        max_p = df[[p1, p2, p3]].max(axis = 1)
        min_p = df[[p1, p2, p3]].min(axis = 1)
        mid_p = df[[p1, p2, p3]].sum(axis = 1) - max_p - min_p
        df[f'{p1}_{p2}_{p3}_imb'] = (max_p - mid_p) / (mid_p - min_p) #normalized p from the 3
        features.append(f'{p1}_{p2}_{p3}_imb')

    return df    
    
    
df = generate_features(df_train)
df.columns

Generating 1st set of features...


15it [00:00, 485.41it/s]


Generating 2st set of features...


20it [00:00, 59.38it/s]


Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'imb_s1', 'imb_s2',
       'reference_price_far_price_imb', 'reference_price_near_price_imb',
       'reference_price_ask_price_imb', 'reference_price_bid_price_imb',
       'reference_price_wap_imb', 'far_price_near_price_imb',
       'far_price_ask_price_imb', 'far_price_bid_price_imb',
       'far_price_wap_imb', 'near_price_ask_price_imb',
       'near_price_bid_price_imb', 'near_price_wap_imb',
       'ask_price_bid_price_imb', 'ask_price_wap_imb', 'bid_price_wap_imb',
       'reference_price_far_price_near_price_imb',
       'reference_price_far_price_ask_price_imb',
       'reference_price_far_price_bid_price_imb',
       'reference_price_far_price_wap_imb',
       'reference_price_near_price_ask_price_imb',
       '

In [6]:
X = df.drop(columns = 'target').values
y = df['target'].values
print(X.shape, y.shape)
X = X[np.isfinite(y)]
y = y[np.isfinite(y)]
idx = np.arange(len(y))
print(X.shape, y.shape, len(idx))

(26455, 53) (26455,)
(26455, 53) (26455,) 26455


In [7]:
model_dict = {
    'lgb': lgb.LGBMRegressor(objective = 'regression_l1', n_estimators = 50),
    'xgb': xgb.XGBRegressor(tree_method = 'hist', objective = 'reg: absoluteerror', n_estimators = 50),
    'cbt': cbt.CatBoostRegressor(objective = 'MAE', iterations = 300)
}


In [3]:
models = []
n_fold = 3
cur_fold = 1
model = lgb.LGBMRegressor(objective = 'regression_l1', n_estimators = 50)
model.fit(X[idx % n_fold != cur_fold],y[idx % n_fold != cur_fold],
          eval_set = [X[idx % n_fold == cur_fold],y[idx % n_fold == cur_fold]])
models.append(model)
models

NameError: name 'lgb' is not defined

In [ ]:
model = model_dict[model_name]
model.fit(X[idx % n_fold != cur_fold], y[idx % n_fold != cur_fold],
        eval_set = [X[idx % n_fold == cur_fold], y[idx % n_fold == cur_fold]])
models.append(model)

In [14]:
def train(model_dict, model_name, cur_fold, n_fold):
    model = model_dict[model_name]
    model.fit(X[idx % n_fold != cur_fold], y[idx % n_fold != cur_fold],
            eval_set = [X[idx % n_fold == cur_fold], y[idx % n_fold == cur_fold]])
    models.append(model)
    joblib.dump(model, f'models/{model_name}_{cur_fold}_model')
    return 

train(model_dict, 'lgb',1, 3)
models    

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002386 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12807
[LightGBM] [Info] Number of data points in the train set: 17637, number of used features: 52


ValueError: Input numpy.ndarray must be 2 dimensional

In [14]:
model_dict = {
    'lgb': lgb.LGBMRegressor(objective = 'regression_l1', n_estimators = 50),
    'xgb': xgb.XGBRegressor(tree_method = 'hist', objective = 'reg: absoluteerror', n_estimators = 50),
    'cbt': cbt.CatBoostRegressor(objective = 'MAE', iterations = 300)
}

X = df.values
y = df['target'].values
X = X[np.isfinite(y)]
y = y[np.isfinite(y)]

idx = np.arange(len(X))

models = []
n_fold = 1

def train(model_dict, model_name, cur_fold, n_fold):
    model = model_dict[model_name]
    model.fit(X[idx % n_fold != cur_fold], y[idx % n_fold != cur_fold],
            eval_set = [X[idx % n_fold == cur_fold], y[idx % n_fold == cur_fold]])
    models.append(model)
    joblib.dump(model, f'models/{model_name}_{cur_fold}_model')
    return 

for cur_fold in tqdm(range(n_fold)):
    train(model_dict, 'lgb', cur_fold, n_fold)

models


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Found array with 0 sample(s) (shape=(0, 54)) while a minimum of 2 is required.

In [2]:
import numpy as np

# Example data
index = np.arange(10)  # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
N_fold = 5

# Let's simulate the cross-validation loop
for i in range(N_fold):
    print(f"Fold {i}:")

    # Calculate the boolean masks
    print(index, N_fold, i)
    train_mask = index % N_fold != i
    test_mask = index % N_fold == i

    # Select training and testing data
    train_indices = index[train_mask]
    test_indices = index[test_mask]

    print("  Training Indices:", train_indices)
    print("  Testing Indices:", test_indices)


Fold 0:
[0 1 2 3 4 5 6 7 8 9] 5 0
  Training Indices: [1 2 3 4 6 7 8 9]
  Testing Indices: [0 5]
Fold 1:
[0 1 2 3 4 5 6 7 8 9] 5 1
  Training Indices: [0 2 3 4 5 7 8 9]
  Testing Indices: [1 6]
Fold 2:
[0 1 2 3 4 5 6 7 8 9] 5 2
  Training Indices: [0 1 3 4 5 6 8 9]
  Testing Indices: [2 7]
Fold 3:
[0 1 2 3 4 5 6 7 8 9] 5 3
  Training Indices: [0 1 2 4 5 6 7 9]
  Testing Indices: [3 8]
Fold 4:
[0 1 2 3 4 5 6 7 8 9] 5 4
  Training Indices: [0 1 2 3 5 6 7 8]
  Testing Indices: [4 9]


In [1]:
print('hello')

hello
